<a href="https://colab.research.google.com/github/MostafaELFEEL/-MostafaELFEEL-s-Project-1-professional-embedded-course-Udacity/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import cv2
import os
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import torchvision
import csv
####################################
def imshow(img):
    plt.imshow(cv2.cvtColor(np.transpose(np.array(img),(1,2,0)),cv2.COLOR_RGB2BGR))
###################################
device = t.device('cuda' if t.cuda.is_available() else 'cpu')

####################################
#load csv and remove the path
with open('/content/drive/MyDrive/pytorch/driving_log.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))
####################################
for i in range(len(data)):
    for j in range(3):
        data[i][j]=data[i][j].replace('C:\\Users\\mosta\\Downloads\\AI racing team\\beta_simulator_windows\\Data\\IMG\\','')
####################################
#load images in data
directory_images="Data\IMG"
print(len(data))
for i in range(len(data)):
    data[i][0]=transforms.ToTensor()(cv2.imread('/content/drive/MyDrive/pytorch/IMG/'+data[i][0]))
        #print(data[i][j].shape)
###################################
#tranform column 3,4,5,6 to tensors
for i in range(len(data)):
    for j in range(3,7):
        data[i][j]=t.tensor(float(data[i][j]))
###################################


class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1=nn.Conv2d(3,64,7)
        self.conv2=nn.Conv2d(64,64,5)
        self.conv3=nn.Conv2d(64,64,3)
        self.pool=nn.MaxPool2d(2,2)


        self.fc1=nn.Linear(64*17*37,128)
        self.fc2=nn.Linear(128,2)

    def forward(self,x):
        # N,3,160,320
        #print(x.shape)
        x=self.pool(F.relu(self.conv1(x)))    # N,64,77,157
        #print(x.shape)

        x=self.pool(F.relu(self.conv2(x)))  # N 64,36,76
        #print(x.shape)

        x=self.pool(F.relu(self.conv3(x)))  # N,64,17,37
        #print(x.shape)

        x=x.view(-1,64*17*37)     # N,64*17*37
        #print(x.shape)

        x=F.relu(self.fc1(x))    # N,128
        #print(x.shape)

        x=self.fc2(x)            # N,2
        #print(x.shape)
        return x
###################################
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        # Define the layers for your CNN architecture
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 40 * 80, 64)  # Adjust the input size based on your image dimensions
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(64, 2)  # Output two values

    def forward(self, x):
        # Define the forward pass of your model
        #N,3,160,320
        x = self.pool1(self.relu1(self.conv1(x))) # N,16,80,160
        x = self.pool2(self.relu2(self.conv2(x))) # N,32,40,80
        x = x.view(-1,32*40*80)  # Flatten the feature maps  #N,32*40*80
        x = self.relu3(self.fc1(x)) # N,64
        x = self.fc2(x) # N,2
        return x

###################################





2629


In [43]:
###################################
#hyperparameters
epochs=20
batch_size=128
learning_rate=0.001   #0.001
#############################
#split data to train,test
#train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
train_data=data
###################################
train_data=t.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
#test_data=t.utils.data.DataLoader(test_data,batch_size=batch_size,shuffle=True)

###################################
#model = ConvNet()
model = CustomCNN().to(device)
#print(model)

###################################

criterion=nn.MSELoss()
optimizer=t.optim.Adam(model.parameters(),lr=learning_rate)

n_total_steps = len(train_data)

###################################

# Train

In [46]:
for epoch in range(epochs):
    running_loss=0.0

    for i,(center,left,right,steering,thrust,reverse,speed) in enumerate(train_data):

        center = center.to(device)


        labels=t.cat((steering.unsqueeze(1),thrust.unsqueeze(1)),dim=1)
        labels = labels.to(device)

        outputs=model(center)
        loss=criterion(outputs,labels)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss+=loss.item()

    print(f'[{epoch + 1}] loss: {running_loss / n_total_steps:.3f}')

print('Finished Training')
PATH = './model.pth'
t.save(model, PATH)


[1] loss: 0.017
[2] loss: 0.016
[3] loss: 0.013
[4] loss: 0.012
[5] loss: 0.011
[6] loss: 0.010
[7] loss: 0.010
[8] loss: 0.009
[9] loss: 0.008
[10] loss: 0.008
[11] loss: 0.007
[12] loss: 0.007
[13] loss: 0.006
[14] loss: 0.006
[15] loss: 0.005
[16] loss: 0.005
[17] loss: 0.006
[18] loss: 0.007
[19] loss: 0.004
[20] loss: 0.005
Finished Training


# Test

In [48]:
mae_total=0
rmse_total=0
counter=0
with t.no_grad():
    for i,(center,left,right,steering,thrust,reverse,speed) in enumerate(test_data):
        labels=t.cat((steering.unsqueeze(1),thrust.unsqueeze(1)),dim=1)
        labels = labels.to(device)
        center = center.to(device)
        outputs=model(center)
        loss=criterion(outputs,labels)

        print(f'[{i + 1}] loss: {loss.item():.3f}')

        mae = t.mean(t.abs(outputs - labels))
        mae_total+=mae.item()


        # Calculate Root Mean Squared Error (RMSE)
        rmse = t.sqrt(t.mean((outputs - labels) ** 2))
        rmse_total+=rmse.item()

        print(f"MAE: {mae.item()}")
        print(f"RMSE: {rmse.item()}")
        counter=i+1


print(f"MAE_total: {mae_total/counter}")
print(f"RMSE_total: {rmse_total/counter}")

[1] loss: 0.012
MAE: 0.08396846055984497
RMSE: 0.1093476340174675
[2] loss: 0.011
MAE: 0.08416697382926941
RMSE: 0.10515771806240082
[3] loss: 0.017
MAE: 0.0946195051074028
RMSE: 0.12902995944023132
[4] loss: 0.013
MAE: 0.09136063605546951
RMSE: 0.11425990611314774
[5] loss: 0.011
MAE: 0.07971160113811493
RMSE: 0.10567068308591843
[6] loss: 0.014
MAE: 0.09189678728580475
RMSE: 0.11815019696950912
[7] loss: 0.015
MAE: 0.09565868973731995
RMSE: 0.12431326508522034
[8] loss: 0.012
MAE: 0.08641223609447479
RMSE: 0.10793934017419815
[9] loss: 0.014
MAE: 0.09387055784463882
RMSE: 0.11674318462610245
[10] loss: 0.009
MAE: 0.07395987212657928
RMSE: 0.09219551831483841
[11] loss: 0.011
MAE: 0.08677958697080612
RMSE: 0.10483354330062866
[12] loss: 0.010
MAE: 0.07705940306186676
RMSE: 0.09798282384872437
[13] loss: 0.010
MAE: 0.07788576185703278
RMSE: 0.10029571503400803
MAE_total: 0.08595000551297115
RMSE_total: 0.10968611446710733
